In [1]:
import zpu
import re

In [2]:
def makeBiseq(delta, theta):
    """Makes one sequence from tje bi-sequence delta andm theta"""
    if len(delta) != len(theta):
            print("délky delta a theta nejsou stejné")
            return 
    s = ""
    for i in range(len(delta)):
        s = s + delta[i] + theta[i]
    return s
def parseBiseq(biseq):
    """Makes from one sequence the bi-sequence delta and theta"""
    delta = ""
    theta = ""
    for i in range(len(biseq)):
        if i%2 == 0:
            delta = delta + biseq[i]
        else:
            theta = theta + biseq[i]
    return [delta, theta]

In [3]:
bad_prefixes = ["(0R)*0E", "(1R)*1E", "(0R)+1E1E", "(1R)+0E0E"]
bad_factors = ["1R0E1E", "1R1E0E", "0R0E1E", "0R1E0E", "1E0R1R", "1E1R0R", "0E0R1R", "0E1R0R"]

In [4]:
def repare_ii(match, a):
    a_bar = ["1", "0"]
    length = match.end()
    s = match.string
    s = s[length:]
    s = a_bar[int(a)] + "E" + s
    s = (a + "R") * int(length/2) + s
    return s
    
def repare_iii(match, a):
    a_bar = ["1", "0"]
    length = match.end()
    s = match.string
    s = s[length:]
    s = a_bar[int(a)] + "E" + a_bar[int(a)] + "R" + a + "E" + s
    s = (a + "R") * int((length-4)/2) + s
    return s
    
def rep_0(match):
    return repare_ii(match, "0")
    
def rep_1(match):
    return repare_ii(match, "1")
    
def rep_2(match):
    return repare_iii(match, "0")
    
def rep_3(match):
    return repare_iii(match, "1")
    
norm_replace_functions = [rep_0, rep_1, rep_2, rep_3]

In [5]:
def Normalize(delta, theta):
    biseq = makeBiseq(delta, theta)
    if biseq.startswith("0R1R"):
        biseq = biseq.replace("0R1R", "0R1E0R", 1)
    elif biseq.startswith("1R0R"):
        biseq = biseq.replace("1R0R", "1R0E1R", 1)
    else:
        i = 0
        matched = re.match(bad_prefixes[i], biseq)
        #print(matched)
        while (i<4) and (matched == None):
            i = i+1
            if i<4:
                matched = re.match(bad_prefixes[i], biseq)
                #print(matched)
            
        if i < 4:
            #print("pattern: " + str(i))
            biseq = norm_replace_functions[i](matched)
    
    i=0
    while (i < len(biseq) - 5):
        j = 0
        while (j <  8) and (biseq[i:i+6] != bad_factors[j]):
            j = j+1
        if (j == 8):
            i = i+1
        else:
            bad = bad_factors[j]
            #print(parseBiseq(bad))
            biseq = biseq.replace(bad, bad[0:5] + bad[1] + bad[2:4], 1)
            #print(parseBiseq(biseq))
            i = max(0, i+2)
    return parseBiseq(biseq)

In [6]:
def isNormalized(delta, theta):
    biseq = makeBiseq(delta, theta)
    if biseq.startswith("0R1R") or biseq.startswith("1R0R"):
        print("je tam RR aa*")
        return False
    elif (re.match(bad_prefixes[0], biseq) != None) or (re.match(bad_prefixes[1], biseq) != None):
        print("je tam R^iE a^i")
        return False
    elif (re.match(bad_prefixes[2], biseq) != None) or (re.match(bad_prefixes[3], biseq) != None):
        print("je tam R^iEE a^ia*a*")
        return False
    elif any(x in biseq for x in bad_factors):
        print("ve slove je tt*t* abb*")
        return False
    else:
        return True

In [7]:
print(isNormalized("010000", "RREEEE"))

je tam RR aa*
False


In [8]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [9]:
def maximizeRinBiseq(delta, theta):
    delta, theta = Normalize(delta, theta)
    biseq = makeBiseq(delta, theta)
    print(biseq)
    badfact = ["0E0R1E0R", "0E1R0E1R", "1E0R1E0R", "1E1R0E1R" ]
    replacement = ["0E0R1R", "0E1R0R", "1E0R1R", "1E1R0R" ]
    l = len(biseq)
    if l >= 8:
        i = l-8
        while (i >= 0):
            j=0
            substring = biseq[i:i+8]
            while (j <  4) and (substring != badfact[j]):
                j = j+1
            if (j == 4):
                i = i-1
            else:
                bad = badfact[j]
                print(parseBiseq(bad))
                biseq = rreplace(biseq, substring, replacement[j],1)
                print(parseBiseq(biseq))
                i = i-2
    if biseq.startswith("1R0E1R"):
        biseq = biseq.replace("0R1E0R", "0R1R", 1)
    elif biseq.startswith("1R0R"):
        biseq = biseq.replace("1R0E1R", "1R0R", 1)
        
    return parseBiseq(biseq)
            

In [10]:
maximizeRinBiseq("000101", "EEERER")

0R1E0R1E0R1E1R0E1R
['1101', 'ERER']
['01010110', 'RERERERR']
['1010', 'ERER']
['0101110', 'RERRERR']


['0101110', 'RERRERR']

In [11]:
p = Normalize("0101010010", "REREREEERR")
print(p)
print(isNormalized(p[0], p[1]))
print("")

r = Normalize("010101010010", "RRREREREEERR")
print(r)
print(isNormalized(r[0], r[1]))

t = Normalize("111000101010010", "RRREEREREREEERR")
print(t)
print(isNormalized(t[0], t[1]))

['0101010101101', 'RERERERERERER']
True

['0100101010101101', 'RERRERERERERERER']
True
['1110010101010101101', 'RRRERERERERERERERER']
True
